# TP Python avancé sur Jupyter

# 🔬 Analyse de la répartition des thèses dans un laboratoire de recherche

Nous disposons de deux fichiers de données :
- `APermanents.csv` : informations sur les chercheurs permanents du laboratoire.
- `PhDData.csv` : informations sur les doctorants encadrés par au moins un permanent.

Les données ont été anonymisées et brouillées. L’objectif est de produire une vue synthétique de la répartition des thèses au sein du laboratoire.

---

## 🧪 Exploration initiale : comportement des copies de DataFrame

- Comparer l’effet d’une affectation directe (`x2 = x`) avec celui d’une copie explicite (`x2 = x.copy()`), en modifiant une ligne du second DataFrame. Observer les impacts sur le DataFrame original.

---

## 📌 Étapes d’analyse

### 1. Chargement des données
- Lire les fichiers `APermanents.csv` et `PhDData.csv` à l’aide de `pandas.read_csv()` et stocker les données dans les DataFrames `Permanents` et `Student`.

### 2. Filtrage selon le statut
- Identifier les chercheurs permanents qui ne sont **pas émérites**.

### 3. Statuts spécifiques sans HDR
- Extraire les chercheurs ayant un **statut DR ou Pr sans HDR** renseignée.

### 4. Encadrants sans HDR
- Identifier les **directeurs de thèse** sans HDR, malgré la règle selon laquelle une HDR est requise (renseignée par une date ou "oui"), en prenant en compte les exceptions.

### 5. Codification du taux d’encadrement par étudiant
- Déterminer et ajouter une **colonne de taux d’encadrement** dans le DataFrame approprié, selon les règles suivantes :
  - Un étudiant avec un seul directeur de thèse compte pour **100%**.
  - Un étudiant avec un directeur et un co-encadrant *ou* une cotutelle (exclusif) compte pour **50%**.
  - Un étudiant avec directeur + co-encadrant + cotutelle compte pour **30%**.

### 6. Taux d’encadrement par permanent
- Pour chaque permanent, **calculer son taux d’encadrement** et l’ajouter au DataFrame.
- Fournir une synthèse : moyenne, écart-type, quantiles, et toute autre mesure pertinente.

### 7. Analyse par équipe
- Pour chaque équipe, calculer :
  - Le **taux d’encadrement maximal** (300% × nombre de permanents),
  - Le **taux moyen** d’encadrement observé,
  - Le **% d’encadrement disponible** (ex. : 550% = possibilité d’encadrer 5,5 thèses).

### 8. Analyse par département
- À partir des regroupements d’équipes en départements, calculer :
  - Le **taux maximal d’encadrement**,
  - Le **taux moyen** par département,
  - Le **% disponible** pour encadrer.

> **Répartition des équipes par département :**
>
> - **Dept1** : Abc, Adagio, Caramba, Gamble, Magrit (alias Tangram, Magrit-post), Mfx, Pixel (alias Alice)  
> - **Dept2** : Carbone, Mocqua, Mosel, Veridis, Pesto, Types, Dedale  
> - **Dept3** : Coast, Optimist, Resist, Simbiot  
> - **Dept4** : Cello, Équipe K, Multispeech, Orpailleur, Read, Smart, Semagramme, Synalp  
> - **Dept5** : Biscuit, Capsid, Kiwi, Larsen, Neurorhythms, Neurosys

### 9. Permanents sans thèse encadrée
- Identifier les chercheurs permanents **n’encadrant aucune thèse**.

### 10. Thèses soutenues ou à soutenir en 2020
- Lister les thèses qui **auraient dû être soutenues ou le seront en 2020**.

### 11. Stratégie de création d’identités fictives
- Élaborer une méthode d’anonymisation garantissant :
  - Le remplacement des **noms et prénoms** réels par des noms/prénoms fictifs tirés aléatoirement de deux fichiers : `ListeDeNoms` (noms) et `Prenoms` (prénoms).
  - La **cohérence entre les fichiers** : un même chercheur ou doctorant doit garder la même identité dans toutes les tables.
  - La **possibilité de refaire les analyses précédentes**, notamment celles qui se basent sur les liens entre doctorants et encadrants.

### 12. Anonymisation des permanents
- Appliquer la stratégie pour anonymiser les identités dans la table des **chercheurs permanents**.

### 13. Anonymisation des doctorants
- Appliquer la stratégie pour anonymiser les identités dans la table des **doctorants**, tout en conservant les correspondances avec les identités anonymisées des encadrants.


## ✅ Résumé des livrables attendus

- Une série de notebooks/documentation illustrant :
  - Les différentes étapes d’analyse (filtrage, calculs, agrégation, visualisation si pertinent),
  - L’ajout de colonnes et de statistiques personnalisées,
  - La mise en place d’un système d’anonymisation reproductible,
  - Une présentation claire et lisible des résultats (markdown, tableaux, commentaires).


### 1. Chargement des données
- Lire les fichiers `APermanents.csv` et `PhDData.csv` à l’aide de `pandas.read_csv()` et stocker les données dans les DataFrames `Permanents` et `Student`.

In [1]:
import pandas as pd
import numpy as np

In [2]:
x=pd.DataFrame.from_dict({'A': [1.1, 2.7, 5.3], 'B': [2, 10, 9],
'C': [3.3, 5.4, 1.5], 'D': [4, 7, 15]}, orient = 'index')
x
x2=x
x2.iloc[0]=[0,0,0]
x

,0,1,2
A,0.0,0.0,0.0
B,2.0,10.0,9.0
C,3.3,5.4,1.5
D,4.0,7.0,15.0


In [3]:
x2

,0,1,2
A,0.0,0.0,0.0
B,2.0,10.0,9.0
C,3.3,5.4,1.5
D,4.0,7.0,15.0


In [4]:
x=pd.DataFrame.from_dict({'A': [1.1, 2.7, 5.3], 'B': [2, 10, 9],
'C': [3.3, 5.4, 1.5], 'D': [4, 7, 15]}, orient = 'index')
x
x2=x.copy()
x2.iloc[0]=[0,0,0]
x

,0,1,2
A,1.1,2.7,5.3
B,2.0,10.0,9.0
C,3.3,5.4,1.5
D,4.0,7.0,15.0


In [5]:
x2

,0,1,2
A,0.0,0.0,0.0
B,2.0,10.0,9.0
C,3.3,5.4,1.5
D,4.0,7.0,15.0


Dans le premier code du Listing 1, on affecte x à x2 et donc les deux dataframes sont les mêmes, du coup si on fait des modifications sur x2, on les fait aussi sur x car les deux dataframes occupent la même localisation dans la mémoire. Ainsi, lors de l'affichage de x, on voit que la première ligne a été remplacée par une ligne de 0 comme indiquée par la commande (x2.iloc[0]=[0,0,0]). Cependant, dans le deuxième code, x2 est une copie de x, elles n'occupent pas le même espace mémoire, et si on fait des modifications sur x2 cela n'a rien à voir avec x car les deux dataframes sont indépendants. C'est pour cette raison qu'on ne voit pas de modifications sur x lors de l'affichage final (la première ligne reste la même) mais qu'on les voit sur x.

### 2. Filtrage selon le statut
- Identifier les chercheurs permanents qui ne sont **pas émérites**.

In [6]:
#Lecture des deux dataframes
file1 = 'Apermanents.csv'
Permanents = pd.read_csv(file1)
file2 = 'PhDData.csv'
Student = pd.read_csv(file2)
#Affichage des deux dataframes
print(Permanents)
print(Student)

    \tNOMEnc\tPRENOMEnc\tEQUIPE\tEmeritat\tHDR\tSTATUT
0       0\tBoisvert\tJohnie\tOptimist\t\t12/12/18\tMcf
1               1\tBaillargeon\tGeorges\tPixel\t\t\tCr
2      2\tMorneau\tEsteve\tSemagramme\t\t28/11/16\tMcf
3                3\tLimoges\tPatton\tOptimist\t\t\tMcf
4              4\tBougie\tTegan\tMosel\t\t25/06/18\tPr
..                                                 ...
202                202\tGiasson\tNereida\tMfx\t\t\tMcf
203   203\tBoisjoli\tLothaire\tCaramba\t\t26/11/01\tDr
204                     204\tBourque\tMayte\t\t\tOui\t
205                     205\tGemme\tIoannes\t\t\tOui\t
206                      206\tBoileau\tJune\t\t\tOui\t

[207 rows x 1 columns]
    \tNom\tPrenom\tNationalité\tEquipe\tInscription en thèse\tSujet de thèse\tNom Dir\tPrenom Dir\tNom Coenc\tPrenom Coenc\tDate début thèse\tdate fin thèse\tCotutelle\tEtab cotutelle\tPays cotutelle\tDate de naissance
0    0\tRancourt\tMarceline\tAlgerienne\tOrpailleur...                                           

#### On effectue des modifications sur les dataframes pour améliorer l'affichage

In [7]:
Permanents = pd.read_csv(file1, usecols = ['NOMEnc', 'PRENOMEnc', 'EQUIPE', 'Emeritat', 'HDR', 'STATUT'], sep = '\t')

Student = pd.read_csv(file2, usecols = ['Nom', 'Prenom', 'Nationalité', 'Equipe', 'Inscription en thèse', 'Sujet de thèse',
                                       'Nom Dir', 'Prenom Dir', 'Nom Coenc', 'Prenom Coenc', 'Date début thèse', 'date fin thèse',
                                       'Cotutelle', 'Etab cotutelle', 'Pays cotutelle', 'Pays cotutelle'], sep = '\t')
Permanents, Student

(          NOMEnc PRENOMEnc      EQUIPE Emeritat       HDR STATUT
 0       Boisvert    Johnie    Optimist      NaN  12/12/18    Mcf
 1    Baillargeon   Georges       Pixel      NaN       NaN     Cr
 2        Morneau    Esteve  Semagramme      NaN  28/11/16    Mcf
 3        Limoges    Patton    Optimist      NaN       NaN    Mcf
 4         Bougie     Tegan       Mosel      NaN  25/06/18     Pr
 ..           ...       ...         ...      ...       ...    ...
 202      Giasson   Nereida         Mfx      NaN       NaN    Mcf
 203     Boisjoli  Lothaire     Caramba      NaN  26/11/01     Dr
 204      Bourque     Mayte         NaN      NaN       Oui    NaN
 205        Gemme   Ioannes         NaN      NaN       Oui    NaN
 206      Boileau      June         NaN      NaN       Oui    NaN
 
 [207 rows x 6 columns],
            Nom     Prenom    Nationalité       Equipe Inscription en thèse  \
 0     Rancourt  Marceline     Algerienne   Orpailleur           15-10-2018   
 1       Tardif    Came

### 3. Statuts spécifiques sans HDR
- Extraire les chercheurs ayant un **statut DR ou Pr sans HDR** renseignée.

#### Tous les encadrants qui ne sont pas émerites ont des NaN dans la colonne Emeritat, on les cible avec la commande .isna()

In [8]:
Permanents[Permanents['Emeritat'].isna()]

,NOMEnc,PRENOMEnc,EQUIPE,Emeritat,HDR,STATUT
0,Boisvert,Johnie,Optimist,NaN,12/12/18,Mcf
1,Baillargeon,Georges,Pixel,NaN,NaN,Cr
2,Morneau,Esteve,Semagramme,NaN,28/11/16,Mcf
3,Limoges,Patton,Optimist,NaN,NaN,Mcf
4,Bougie,Tegan,Mosel,NaN,25/06/18,Pr
...,...,...,...,...,...,...
202,Giasson,Nereida,Mfx,NaN,NaN,Mcf
203,Boisjoli,Lothaire,Caramba,NaN,26/11/01,Dr
204,Bourque,Mayte,NaN,NaN,Oui,NaN
205,Gemme,Ioannes,NaN,NaN,Oui,NaN


#### on peut également à titre informatif afficher tous les permanents qui sont émérites

In [9]:
Permanents[Permanents['Emeritat']=='Emeritat']

,NOMEnc,PRENOMEnc,EQUIPE,Emeritat,HDR,STATUT
88,Plouffe,Armin,Dept5,Emeritat,NaN,Pr
94,Bourdeau,Moana,Dept4,Emeritat,NaN,Pr
95,Poissant,Giedre,Dept4,Emeritat,NaN,NaN
127,Brunelle,Uriah,Capsid,Emeritat,NaN,Dr16
129,Bolduc,Wilda,Orpailleur,Emeritat,28/10/96,Pr
142,Turbide,Innocenzo,Orpailleur,Emeritat,31/12/92,Dr
156,Martel,Chiarina,Semagramme,Emeritat,12/11/03,Pr
172,Surprenant,Danut,Dept3,Emeritat,NaN,Pr
176,Provost,Kimberly,Dept3,Emeritat,NaN,Pr
188,Rioux,Lizbeth,Dept4,Emeritat,NaN,Pr


### 4. Encadrants sans HDR
- Identifier les **directeurs de thèse** sans HDR, malgré la règle selon laquelle une HDR est requise (renseignée par une date ou "oui"), en prenant en compte les exceptions.

#### On veut lister tous les permanents de statut 'Dr' ou 'Pr' et dont la mention 'HDR' n'est pas renseignée. On s'intéresse donc aux colonnes "STATUT" et "HDR", qui dans ce cas là vaudront respectivement : 'Dr' ou 'Pr' et NaN.

In [10]:
Permanents[(Permanents['STATUT'].isin(['Dr','Pr'])) & (Permanents['HDR'].isna())]

,NOMEnc,PRENOMEnc,EQUIPE,Emeritat,HDR,STATUT
46,Damours,Nicolette,Loria-dir,NaN,NaN,Pr
52,Goudreau,Lisha,Orpailleur,NaN,NaN,Pr
57,Goyer,Sroel,Semagramme,NaN,NaN,Dr
77,Pierre,Alessio,Types,NaN,NaN,Pr
87,Bourgeois,Jill,Coast,NaN,NaN,Pr
88,Plouffe,Armin,Dept5,Emeritat,NaN,Pr
94,Bourdeau,Moana,Dept4,Emeritat,NaN,Pr
96,Riendeau,Anto,Neurosys loria,NaN,NaN,Pr
144,Viens,Oksana,Simbiot,NaN,NaN,Pr
172,Surprenant,Danut,Dept3,Emeritat,NaN,Pr


### 5. Codification du taux d’encadrement par étudiant
- Déterminer et ajouter une **colonne de taux d’encadrement** dans le DataFrame approprié, selon les règles suivantes :
  - Un étudiant avec un seul directeur de thèse compte pour **100%**.
  - Un étudiant avec un directeur et un co-encadrant *ou* une cotutelle (exclusif) compte pour **50%**.
  - Un étudiant avec directeur + co-encadrant + cotutelle compte pour **30%**.


#### On souhaite lister tous les directeurs de thèse qui n'ont pas de HDR. 

##### Un directeur de thèse est un permanent qui encadre aussi un étudiant. Cet individu devra donc apparaitre dans les deux dataframes : dans Permanents sous la colonne "NOMEnc" et dans Student sous la colonne "Nom Dir". 
##### Puisque ces directeurs de thèse n'ont pas de HDR, la colonne "HDR" aura donc la mention NaN.

In [11]:
Permanents[(Permanents['HDR'].isna()) & (Permanents['NOMEnc'].isin(Student['Nom Dir']))]

,NOMEnc,PRENOMEnc,EQUIPE,Emeritat,HDR,STATUT
52,Goudreau,Lisha,Orpailleur,NaN,NaN,Pr
57,Goyer,Sroel,Semagramme,NaN,NaN,Dr
59,Boivin,Teddy,Capsid,NaN,NaN,Cr
96,Riendeau,Anto,Neurosys loria,NaN,NaN,Pr


### 6. Taux d’encadrement par permanent
- Pour chaque permanent, **calculer son taux d’encadrement** et l’ajouter au DataFrame.
- Fournir une synthèse : moyenne, écart-type, quantiles, et toute autre mesure pertinente.

#### Dans cette question, ainsi que dans la question suivante on s'intéresse au calcul du taux d'encadrement des chercheurs, qui est égal à :
##### 30 si l'étudiant bénéficie d'un directeur de thèse, d'un coencadrant et d'une cotutelle 
##### 50 si l'étudiant bénéficie d'un directeur de thèse et d'un coencadrant 
##### 50 si l'étudiant bénéficie d'un directeur de thèse et d'une cotutelle 
##### 100 si l'étudiant bénéficie uniquement d'un directeur de thèse

#### On décide de coder cette information dans une colonne 'taux' qu'on rajoute au dataframe 'Student', puisque c'est dans ce dataframe qu'on retrouve les informations sur l'encadrement, le coencadrement et la cotutelle et c'est donc plus simple à calculer.

#### La colonne 'taux' représente le taux d'encadrement de chacun des étudiants par les diférents chercheurs.

In [12]:
Student['taux']=100
Student.loc[(Student['Nom Coenc'].notna()) & (Student['Cotutelle']=='Non'),'taux'] = 50
Student.loc[(Student['Nom Coenc'].isna()) & (Student['Cotutelle']=='Oui'),'taux'] = 50
Student.loc[(Student['Nom Coenc'].notna()) & (Student['Cotutelle']=='Oui'),'taux'] = 30
Student[['taux']]

,taux
0,100
1,50
2,100
3,50
4,50
...,...
152,100
153,100
154,50
155,50


### 7. Analyse par équipe
- Pour chaque équipe, calculer :
  - Le **taux d’encadrement maximal** (300% × nombre de permanents),
  - Le **taux moyen** d’encadrement observé,
  - Le **% d’encadrement disponible** (ex. : 550% = possibilité d’encadrer 5,5 thèses).

#### Afin de calculer le taux d'encadrement de chaque permanent, qu'il soit coencadrant ou pas, on s'intérese à la colonne taux de Student. On crée deux noveaux dataframes qui contiennent le nom et prénom du directeur(le cas échéant, du coencadrant) extraits de Student, ainsi, que la colonne taux. Après avoir nettoyé les deux dataframes, on renomme les colonnes : 'nom' et 'prénom' pour avoir des structures similaires afin d'effectuer une jointure.On les concatène en bas et on obtient le dataframe "Perm" qui contient les taux d'encadrement de tous les encadrants mais qui peut aussi contenir des doublons.

In [13]:
#on crée deux nouveaux dataframes
StudentDir = Student[['Nom Dir','Prenom Dir', 'taux']].copy()
StudentCoenc = Student[['Nom Coenc','Prenom Coenc', 'taux']].copy()

#on enlève les NAN des coencadrants
Student = Student[Student['Nom Coenc'].notna()]

#on renomme les colonnes
StudentDir.rename(columns={'Nom Dir': 'Nom', 'Prenom Dir': 'Prenom'}, inplace=True)
StudentCoenc.rename(columns={'Nom Coenc': 'Nom', 'Prenom Coenc': 'Prenom'}, inplace=True)

#on affiche les data frames 
StudentDir
StudentCoenc

,Nom,Prenom,taux
0,NaN,NaN,100
1,Denis,Jacques,50
2,NaN,NaN,100
3,Blackburn,Gertraud,50
4,NaN,NaN,50
...,...,...,...
152,NaN,NaN,100
153,NaN,NaN,100
154,Robertson,Bertie,50
155,Jones,Ning,50


In [14]:
#on concatène les deux dataframes
Perm = pd.concat([StudentDir, StudentCoenc])
Perm


,Nom,Prenom,taux
0,Turbide,Innocenzo,100
1,Daoust,Ulyssa,50
2,Durocher,Neilina,100
3,Rose,Macario,50
4,Rancourt,Sans,50
...,...,...,...
152,NaN,NaN,100
153,NaN,NaN,100
154,Robertson,Bertie,50
155,Jones,Ning,50


#### Dans cette partie de la question, on souhaite calculer le taux d'encadrement de chaque équipe selon le nombre d'étudiants encadrés: 
#### On extrait d'abord les équipes et on les stocke dans un nouveau dataframe "Perm1", et on effectue un "merge" entre les deux df : "Perm" et "Perm1" qu'on associe à un nouveau dataframe : "Permfull". On signale qu'à ce stade là, Permfull continet encore des doublons mais c'est nécessaire pour la continuité du calcul.
#### Ensuite, pour avoir tous les encadrants par équipe, on utilise la commande "GroupBy", qui permet de grouper les individus par équipe. En associant cette commende à l'aggrégation (.agg), on calcule le taux d'encadrement total, ainsi que le nombre d'étudiants par équipe puisque l'encadrant apparait dans le dataframe Permfull autant de fois qu'il encadre d'étudiants donc en effectuant la somme des lignes répétées par encadrant on parvient au résultat souhaité.
#### On stocke le résultat du groupby dans un df qui s'intitule : "p" et on effectue quelques modifications pour englober les permanents qui n'encadrent pas d'étudiants (dont le taux = 0 et le nbr =1 puisqu'ils apparaissent au moins une fois dans le dataframe issu du merge).

In [15]:
#on extrait les equipes de Permanents dans un nouveau DataFrame
Perm1=Permanents[['NOMEnc','PRENOMEnc','EQUIPE']].copy()
#on renomme les colonnes
Perm1.rename(columns={'NOMEnc': 'Nom', 'PRENOMEnc': 'Prenom'}, inplace=True)
#on merge les deux dataframes pour avoir les équipes
Permfull = pd.merge(Perm, Perm1, how="outer", on=["Nom", "Prenom"])
#on group by pour avoir le taux d'encadrement et le nbr d'étudiants par encadrant
p=Permfull.groupby(['Nom','Prenom','EQUIPE']).agg(nbr=('taux','size'),total=('taux','sum'))
#pour les encadrants qui n'ont pas d'étudiants et qui ont que des NAN dans le merge
p.loc[(p['total']== 0.0) & (p['nbr']== 1),'nbr'] = 0
#on affiche le résultat
p




,,,nbr,total
Nom,Prenom,EQUIPE,,
Ahmed,Morris,Coast,3,150.0
Anctil,Annikki,Semagramme,0,0.0
April,Ellie,Resist,1,50.0
Arsenault,Georgo,Types,0,0.0
Audy,Kaden,Synalp,0,0.0
...,...,...,...,...
Villeneuve,Tracy,Mosel,1,50.0
Vinet,Tiras,Mocqua,3,150.0
Walker,Adisa,Capsid,2,100.0


#### Moyenne, écart-type et quantiles sur les permanents

In [92]:
p.describe()

,nbr,total
count,203.000000,203.000000
mean,1.192118,74.088670
std,1.569365,105.673381
min,0.000000,0.000000
25%,0.000000,0.000000
50%,1.000000,50.000000
75%,2.000000,100.000000
max,7.000000,600.000000


### 8. Analyse par département
- À partir des regroupements d’équipes en départements, calculer :
  - Le **taux maximal d’encadrement**,
  - Le **taux moyen** par département,
  - Le **% disponible** pour encadrer.

> **Répartition des équipes par département :**
>
> - **Dept1** : Abc, Adagio, Caramba, Gamble, Magrit (alias Tangram, Magrit-post), Mfx, Pixel (alias Alice)  
> - **Dept2** : Carbone, Mocqua, Mosel, Veridis, Pesto, Types, Dedale  
> - **Dept3** : Coast, Optimist, Resist, Simbiot  
> - **Dept4** : Cello, Équipe K, Multispeech, Orpailleur, Read, Smart, Semagramme, Synalp  
> - **Dept5** : Biscuit, Capsid, Kiwi, Larsen, Neurorhythms, Neurosys

#### Dans cette question, on s'intéresse au calcul de quelques mesures par équipe :
#### - le taux d'encadrement réel
#### - le taux d'encadrement moyen
#### - le taux d'encadrement maximal = 300% * nbr de permanents
#### - le pourcentage disponible par équipe, qui sera donc égal au : taux d'encadrement maximal - taux d'encadrement réel
#### Pour ce faire, on crée un nouveau df : "tauxEncEquipe" dans lequel on groupe par 'Equipe' et on effectue les calculs grâce à la fonctionnalité .agg qu'on applique à la colonne 'total'. Pour le total réel; on utilise la fonction ('sum'), pour le total moyen; on utilise la fonciton 'mean', et pour le total maximal on effectue une simple multiplication. Le total disponible est également donné par une simple soustraction.

In [93]:
tauxEncEquipe = p.groupby('EQUIPE').agg(total_reel_equipe=('total','sum'),total_moy_equipe=('total','mean'),total_max_equipe=('total','size'))
tauxEncEquipe['total_max_equipe']=tauxEncEquipe['total_max_equipe']*300
tauxEncEquipe['total_dispo_equipe']=tauxEncEquipe['total_max_equipe']-tauxEncEquipe['total_reel_equipe']
tauxEncEquipe

,total_reel_equipe,total_moy_equipe,total_max_equipe,total_dispo_equipe
EQUIPE,,,,
Abc,50.0,16.666667,900,850.0
Adagio,200.0,40.000000,1500,1300.0
Alice-post,0.0,0.000000,300,300.0
At-lor ae,100.0,50.000000,600,500.0
Biscuit,400.0,100.000000,1200,800.0
Capsid,650.0,130.000000,1500,850.0
Caramba,500.0,55.555556,2700,2200.0
Carbone,300.0,150.000000,600,300.0
Cello,150.0,150.000000,300,150.0


### 9. Permanents sans thèse encadrée
- Identifier les chercheurs permanents **n’encadrant aucune thèse**.

### Comme les équipes sont regoupées par département, on veut calculer le taux d'encadrement de chaque département. On commence d'abord par rajouter une colonne 'DEPARTEMENT' qu'on initialise à 'Dept', ensuite on se réfère à la liste donnée en énoncé pour associer chaque équipe à son département respectif avec la commande ".loc".

In [17]:
#on crée une nouvelle colonne opur les départements
tauxEncEquipe['DEPARTEMENT'] = 'Dept'
#
tauxEncEquipe.reset_index(inplace=True)
#on affecte les départements aux équipes
tauxEncEquipe.loc[(tauxEncEquipe['EQUIPE'].isin(['Abc', 'Adagio', 'Caramba', 'Gamble', 'Magrit' ,'Magrit-post', 'Mfx', 'Pixel','Alice-post'])
) ,'DEPARTEMENT'] = 'Dept1'
tauxEncEquipe.loc[(tauxEncEquipe['EQUIPE'].isin(['Carbone', 'Mocqua', 'Mosel', 'Veridis', 'Pesto', 'Types', 'Dedale'])
) ,'DEPARTEMENT'] = 'Dept2'
tauxEncEquipe.loc[(tauxEncEquipe['EQUIPE'].isin(['Coast', 'Optimist', 'Resist', 'Simbiot'])
) ,'DEPARTEMENT'] = 'Dept3'
tauxEncEquipe.loc[(tauxEncEquipe['EQUIPE'].isin(['Cello', 'Équipe K', 'Multispeech', 'Orpailleur', 'Read', 'Smart', 'Semagramme', 'Syna'])
) ,'DEPARTEMENT'] = 'Dept4'
tauxEncEquipe.loc[(tauxEncEquipe['EQUIPE'].isin(['Biscuit', 'Capsid' , 'Kiwi', 'Larsen', 'Neurorhythms', 'Neurosys'])
) ,'DEPARTEMENT'] = 'Dept5'

tauxEncEquipe

,EQUIPE,total_reel_equipe,total_moy_equipe,total_max_equipe,total_dispo_equipe,DEPARTEMENT
0,Abc,50.0,16.666667,900,850.0,Dept1
1,Adagio,200.0,40.000000,1500,1300.0,Dept1
2,Alice-post,0.0,0.000000,300,300.0,Dept1
3,At-lor ae,100.0,50.000000,600,500.0,Dept
4,Biscuit,400.0,100.000000,1200,800.0,Dept5
5,Capsid,650.0,130.000000,1500,850.0,Dept5
6,Caramba,500.0,55.555556,2700,2200.0,Dept1
7,Carbone,300.0,150.000000,600,300.0,Dept2
8,Cello,150.0,150.000000,300,150.0,Dept4
9,Coast,700.0,70.000000,3000,2300.0,Dept3


### 10. Thèses soutenues ou à soutenir en 2020
- Lister les thèses qui **auraient dû être soutenues ou le seront en 2020**.

### On veut la liste des permanents qui n'encadrent aucune thèse, ils ont donc un nombre d'étudiants encadrés nul. On effectue une simple recherche dans le tableau et on affiche que les lignes pour lesquelles la collone : 'nbr' est égale à 0.

### NB : il faut décommenter une ligne dans le cellule suivante avant de l'exécuter. Merci de la remettre en commentaire après l'exécution.

In [94]:
#décommenter la ligne suivante pour exécuter la cellule 

#p.reset_index(inplace=True)
p[p['nbr']== 0]


,Nom,Prenom,EQUIPE,nbr,total
1,Anctil,Annikki,Semagramme,0,0.0
3,Arsenault,Georgo,Types,0,0.0
4,Audy,Kaden,Synalp,0,0.0
5,Auger,Madog,Kiwi,0,0.0
6,Baillargeon,Georges,Pixel,0,0.0
...,...,...,...,...,...
193,Turmel,Jin,Coast,0,0.0
195,Verret,Finbar,Dedale,0,0.0
196,Verville,Lionel,Caramba,0,0.0
197,Viens,Oksana,Simbiot,0,0.0


## 🔐 Anonymisation des données

### 11. Stratégie de création d’identités fictives
- Élaborer une méthode d’anonymisation garantissant :
  - Le remplacement des **noms et prénoms** réels par des noms/prénoms fictifs tirés aléatoirement de deux fichiers : `ListeDeNoms` (noms) et `Prenoms` (prénoms).
  - La **cohérence entre les fichiers** : un même chercheur ou doctorant doit garder la même identité dans toutes les tables.
  - La **possibilité de refaire les analyses précédentes**, notamment celles qui se basent sur les liens entre doctorants et encadrants.

### Dans cette question, on souhaite lister toutes les thèses qui étaient soutenues en 2020. On importe donc le module datetime et on convertit la colonne 'date fin thèse' au format approprié qui pourrait être ensuite utilisé par datetime. On crée deux nouvelles colonnes contenant la date de début et la date de fin de l'année 2020 et on effectue une simple sélection sur les élèves dont la date de fin de thèse est après la date de début de 2020, et avant la date de fin de 2020. 

In [19]:
import datetime


Student['date fin thèse'] = pd.to_datetime(Student['date fin thèse'],format="%d-%m-%Y")

datedebut = datetime.datetime.strptime('01-01-2020', "%d-%m-%Y")
datefin = datetime.datetime.strptime('31-12-2020', "%d-%m-%Y")

Student[(Student['date fin thèse'] >= datedebut) & (Student['date fin thèse'] <= datefin) ]




C:\Users\Utilisateur\AppData\Local\Temp\ipykernel_20184\671132674.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Student['date fin thèse'] = pd.to_datetime(Student['date fin thèse'],format="%d-%m-%Y")


,Nom,Prenom,Nationalité,Equipe,Inscription en thèse,Sujet de thèse,Nom Dir,Prenom Dir,Nom Coenc,Prenom Coenc,Date début thèse,date fin thèse,Cotutelle,Etab cotutelle,Pays cotutelle,taux
3,Petit,Salvatore,Algerienne,Coast,03-04-2017,Ceci est le sujet 3,Rose,Macario,Blackburn,Gertraud,01-03-2017,2020-02-29,Non,NaN,NaN,50
23,Fleurant,Ursella,Francaise,Kiwi,04-12-2017,Ceci est le sujet 23,Jones,Ning,Garon,Lotus,20-11-2017,2020-11-19,Non,NaN,NaN,50
27,Boisjoli,Ximun,Francaise,Semmagramme,26-09-2017,Ceci est le sujet 27,Morneau,Esteve,Goyer,Sroel,01-09-2017,2020-08-31,Non,NaN,NaN,50
43,Cusson,Grisha,Francaise,Orpailleur,01-12-2016,Ceci est le sujet 43,Pereira,Anderson,Goudreau,Lisha,01-12-2016,2020-06-14,Non,NaN,NaN,50
44,Auger,Vonda,Francaise,Resist,27-03-2017,Ceci est le sujet 44,Parent,Ustinya,Letendre,Huppert,01-04-2017,2020-03-31,Non,NaN,NaN,50
88,Canuel,Curtis,Syrienne,NaN,10-01-2018,Ceci est le sujet 88,Guimond,Anselm,Mongrain,Mariko,01-11-2017,2020-10-31,Non,NaN,NaN,50
97,Ahmed,Kweku,Francaise,Mosel-veridis,19-10-2017,Ceci est le sujet 97,Deshaies,Baldric,Dumont,Lauressa,01-10-2017,2020-09-30,Non,NaN,NaN,50
101,Chamberland,Gaenor,Francaise,Semmagramme,04-09-2017,Ceci est le sujet 101,Goyer,Sroel,Busque,Kaylee,01-09-2017,2020-08-31,Non,NaN,NaN,50
102,Lacoste,Nuo,Chinoise,Resist,06-12-2017,Ceci est le sujet 102,Lecompte,Lacy,Denis,Jacques,01-12-2017,2020-11-30,Non,NaN,NaN,50
107,Marcoux,Perele,Francaise,Larsen,18-09-2017,Ceci est le sujet 107,Brisson,Melvin,Gill,Calanthe,01-09-2017,2020-08-31,Non,NaN,NaN,50


### 12. Anonymisation des permanents
- Appliquer la stratégie pour anonymiser les identités dans la table des **chercheurs permanents**.

### Stratégie de création des nouvelles identités :
#### - On importe les noms et prénoms "fake" contenus dans les fichiers csv dans deux nouveaux dataframes : Noms & Prenoms
#### - On crée deux dataframes de refèrence, dans lesquelles on stocke les vrais noms et prénoms, et les noms et prénoms "fake".
#### - On mélange grâce à la fonctionnalité "sample" pour ne pas conserver l'ordre.
#### - On fait un merge entre le df intial : "Permanents", et le df de reférence : "PermREF pour cibler les vrais noms et prenoms, cela rajouterait les colonnes des noms & prénoms anonymes (ou fake). Le résultat du merge donnera notre df anonymisé : PermanentsANONYME
#### - On remplace les colonnes des vrais noms et prénoms par les noms et prénoms anonymes.
#### - On supprime les colonnes de trop (qui dans ce cas contiendront les doublons des noms et prénoms anonymes.
#### NB : On peut aussi renommer les colonnes, changer l'ordre et supprimer les colonnes en trop. Cela a été fait pour les étudiants, pour la création du df StudentANONYME.

### 13. Anonymisation des doctorants
- Appliquer la stratégie pour anonymiser les identités dans la table des **doctorants**, tout en conservant les correspondances avec les identités anonymisées des encadrants.

##### Lecture des deux dataframes

In [20]:
file3 = 'ListeDeNomsSsAccent.csv'
file4 = 'ListeDePrenoms.csv'
Noms = pd.read_csv(file3, usecols = ['Rang-Fréquence', 'Nom de famille', 'Pourcentage'], sep = '\t')
Prenoms = pd.read_csv(file4, usecols = ['Prenom', 'Genre', 'Langage', 'Frequence'], sep = ';')

##### Affichage des deux dataframes

In [21]:
Noms

,Rang-Fréquence,Nom de famille,Pourcentage
0,1,Tremblay,"1,076"
1,2,Gagnon,"0,790"
2,3,Roy,"0,753"
3,5,Bouchard,"0,530"
4,6,Gauthier,"0,522"
...,...,...,...
846,995,Juteau,"0,016"
847,996,Forcier,"0,016"
848,997,Pinsonneault,"0,016"
849,998,Hervieux,"0,016"


In [22]:
Prenoms

,Prenom,Genre,Langage,Frequence
0,aapeli,m,finnish,0.0
1,aapo,m,finnish,0.0
2,aaren,"m,f",english,0.0
3,aarne,m,finnish,0.0
4,aatami,m,finnish,0.0
...,...,...,...,...
8289,zvi,m,jewish,0.1
8290,zvonimir,m,croatian,0.0
8291,zvonimira,f,croatian,0.0
8292,zvonko,m,croatian,0.0


#### On crée un nouveau dataframe pour les permanents avec les prénoms/noms anonymes mélangés et les prénoms/noms d'encadrants

##### Dans PermREF : NOMEnc & PRENOMEnc sont les VRAIS noms et prénoms des encadrants 

In [30]:
PermREF = Noms[['Nom de famille']].copy()
PermREF['Prenom'] = Prenoms['Prenom']

PermREF = PermREF.sample(frac=1).reset_index().drop(['index'],axis=1)

PermREF['NOMEnc'] = Permanents['NOMEnc']
PermREF['PRENOMEnc'] = Permanents['PRENOMEnc']

PermREF=PermREF.dropna()
PermREF

,Nom de famille,Prenom,NOMEnc,PRENOMEnc
0,Moreau,afon,Boisvert,Johnie
1,Pichette,angelos,Baillargeon,Georges
2,Rodriguez,apolinar,Morneau,Esteve
3,Guay,adorinda,Limoges,Patton
4,Alarie,annabelle,Bougie,Tegan
...,...,...,...,...
202,Duguay,aletha,Giasson,Nereida
203,Sauriol,bernardetta,Boisjoli,Lothaire
204,Loyer,auguste,Bourque,Mayte
205,Sylvain,anushka,Gemme,Ioannes


#### On merge les deux bases de données Permanents et PermREF

In [38]:
PermanentsANONYME = pd.merge(Permanents, PermREF, how="outer", on=["NOMEnc", "PRENOMEnc"])
PermanentsANONYME

,NOMEnc,PRENOMEnc,EQUIPE,Emeritat,HDR,STATUT,Nom de famille,Prenom
0,Boisvert,Johnie,Optimist,NaN,12/12/18,Mcf,Moreau,afon
1,Baillargeon,Georges,Pixel,NaN,NaN,Cr,Pichette,angelos
2,Morneau,Esteve,Semagramme,NaN,28/11/16,Mcf,Rodriguez,apolinar
3,Limoges,Patton,Optimist,NaN,NaN,Mcf,Guay,adorinda
4,Bougie,Tegan,Mosel,NaN,25/06/18,Pr,Alarie,annabelle
...,...,...,...,...,...,...,...,...
202,Giasson,Nereida,Mfx,NaN,NaN,Mcf,Duguay,aletha
203,Boisjoli,Lothaire,Caramba,NaN,26/11/01,Dr,Sauriol,bernardetta
204,Bourque,Mayte,NaN,NaN,Oui,NaN,Loyer,auguste
205,Gemme,Ioannes,NaN,NaN,Oui,NaN,Sylvain,anushka


#### On remplace les colonnes contenant les noms et prénoms de encadrants qui ne sont pas anonymisés par les colonnes anonymisées

In [43]:
PermanentsANONYME['NOMEnc']=PermanentsANONYME['Nom de famille']
PermanentsANONYME['PRENOMEnc']=PermanentsANONYME['Prenom']

PermanentsANONYME.drop(columns=['Nom de famille', 'Prenom'], inplace=True, axis=1)

,NOMEnc,PRENOMEnc,EQUIPE,Emeritat,HDR,STATUT
0,Moreau,afon,Optimist,NaN,12/12/18,Mcf
1,Pichette,angelos,Pixel,NaN,NaN,Cr
2,Rodriguez,apolinar,Semagramme,NaN,28/11/16,Mcf
3,Guay,adorinda,Optimist,NaN,NaN,Mcf
4,Alarie,annabelle,Mosel,NaN,25/06/18,Pr
...,...,...,...,...,...,...
202,Duguay,aletha,Mfx,NaN,NaN,Mcf
203,Sauriol,bernardetta,Caramba,NaN,26/11/01,Dr
204,Loyer,auguste,NaN,NaN,Oui,NaN
205,Sylvain,anushka,NaN,NaN,Oui,NaN


#### On affiche le dataframe des permanents anonymisé

##### Dans PermanentsANONYME : NOMEnc et PRENOMEnc sont les noms et prénoms ANONYMES des encadrants, et non pas les vrais prénoms Pour trouver les vrais noms et prénoms, il faut se reférer au dataframe PermREF:
##### - reporter les colonnes NOMEnc, PRENOMEnc de PermanentsANONYME sur les colonnes Nom de famille, Prenom de PermREF 
##### - regarder les colonnes NOMEnc, PRENOMEnc de PermREF qui ellent contiennent les vrais noms & prénoms

##### J'ai choisi de garder les noms de colonnes comme dans la vraie base de données Permanents, même si ce n'est pas le plus judicieux puisque cela pourrait porter à confusion lors de la comparaison avec le dataframe de reférence.

In [88]:
PermanentsANONYME

,NOMEnc,PRENOMEnc,EQUIPE,Emeritat,HDR,STATUT
0,Moreau,afon,Optimist,NaN,12/12/18,Mcf
1,Pichette,angelos,Pixel,NaN,NaN,Cr
2,Rodriguez,apolinar,Semagramme,NaN,28/11/16,Mcf
3,Guay,adorinda,Optimist,NaN,NaN,Mcf
4,Alarie,annabelle,Mosel,NaN,25/06/18,Pr
...,...,...,...,...,...,...
202,Duguay,aletha,Mfx,NaN,NaN,Mcf
203,Sauriol,bernardetta,Caramba,NaN,26/11/01,Dr
204,Loyer,auguste,NaN,NaN,Oui,NaN
205,Sylvain,anushka,NaN,NaN,Oui,NaN


### 14 :

#### On construit le dataframe de reférence pour Student de la même manière que pour Permanents

In [45]:
#on sélectionne les colonnes de nom et prénom dans les datframes anonymes
StudentREF = Noms[['Nom de famille']].copy()
StudentREF['PrenomANO_ETU'] = Prenoms['Prenom']

#on renomme la colonne nom de famille
StudentREF.rename(columns={'Nom de famille': 'NomANO_ETU'}, inplace=True)

#on mélange les noms et prénoms
StudentREF = StudentREF.sample(frac=1).reset_index().drop(['index'],axis=1)

#on rajoute les vrais noms et prénoms des étudiants
StudentREF['NomETU'] = Student['Nom']
StudentREF['PrenomETU'] = Student['Prenom']


StudentREF=StudentREF.dropna()

StudentREF.rename(columns={'NomETU': 'Nom','PrenomETU': 'Prenom'}, inplace=True)

StudentREF

,NomANO_ETU,PrenomANO_ETU,Nom,Prenom
1,Gauthier,aatami,Tardif,Camelia
3,Harvey,agrafena,Petit,Salvatore
5,Jutras,amyas,Bourdon,Talia
6,Arsenault,adisa,Miron,Sam
9,Dion,adelaide,Gonzalez,Aatto
...,...,...,...,...
150,Brunet,aintzane,Morel,Wenzel
151,Ferland,alejo,Brouillette,Danut
154,Turmel,areli,Lefebvre,Bosede
155,Messier,amato,Rochefort,Victoire


##### ici, afin de ne pas porter à confusion, on nomme les colonnes de noms et prénoms "fake" : "NomANO_ETU" et "PrenomANO_ETU", tandis que les vrais noms et prénoms sont ici représentés par : "Nom" et "Prenom"

### On merge les deux dataframes Student et StudentREF : le dataframe initial et le dataframe de référence sur les noms et prénoms communs

In [72]:
StudentANO = pd.merge(Student, StudentREF, how="outer", on=["Nom", "Prenom"])
StudentANO

,Nom,Prenom,Nationalité,Equipe,Inscription en thèse,Sujet de thèse,Nom Dir,Prenom Dir,Nom Coenc,Prenom Coenc,Date début thèse,date fin thèse,Cotutelle,Etab cotutelle,Pays cotutelle,taux,NomANO_ETU,PrenomANO_ETU
0,Tardif,Camelia,Libanaise,Pesto,05-09-2018,Ceci est le sujet 1,Daoust,Ulyssa,Denis,Jacques,20-08-2018,2021-08-19,Non,NaN,NaN,50,Gauthier,aatami
1,Petit,Salvatore,Algerienne,Coast,03-04-2017,Ceci est le sujet 3,Rose,Macario,Blackburn,Gertraud,01-03-2017,2020-02-29,Non,NaN,NaN,50,Harvey,agrafena
2,Bourdon,Talia,Iranienne,Pesto,22-10-2018,Ceci est le sujet 5,Daoust,Ulyssa,Olivier,Kasandra,01-10-2018,2021-09-30,Non,NaN,NaN,50,Jutras,amyas
3,Miron,Sam,Bresilienne,Orpailleur,14-11-2018,Ceci est le sujet 6,Goudreau,Lisha,Turbide,Innocenzo,01-11-2018,2021-10-31,Non,NaN,NaN,50,Arsenault,adisa
4,Gonzalez,Aatto,Ukrainienne,Neurorhythms,06-06-2018,Ceci est le sujet 9,Riendeau,Anto,Campbell,Callie,01-10-2017,2019-08-31,Oui,The national technical university of ukraine,Ukraine,30,Dion,adelaide
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Morel,Wenzel,Chinoise,Larsen,07-10-2019,Ceci est le sujet 150,Couillard,Venceslao,Lheureux,Karina,01-10-2019,2022-09-30,Non,NaN,NaN,50,Brunet,aintzane
90,Brouillette,Danut,Marocaine,Resist,28-10-2019,Ceci est le sujet 151,Lecompte,Lacy,Boulianne,Benaiah,01-10-2019,2022-09-30,Oui,Universite internationale de rabat,Maroc,30,Ferland,alejo
91,Lefebvre,Bosede,Grecque,Orpailleur,24-01-2020,Ceci est le sujet 154,Goudreau,Lisha,Robertson,Bertie,01-11-2019,2022-10-31,Non,NaN,NaN,50,Turmel,areli
92,Rochefort,Victoire,Chinoise,Kiwi,31-10-2018,Ceci est le sujet 155,Caron,Breda,Jones,Ning,01-10-2018,2021-09-30,Non,NaN,NaN,50,Messier,amato


#### On remplace les colonnes contenant les noms et prénoms des étudiants qui ne sont pas anonymisés par les colonnes anonymisées

In [73]:
#on attribue aux colonnes noms et prénoms des élèves les nomes et prénoms anonymes

StudentANO['Nom']=StudentANO['NomANO_ETU']
StudentANO['Prenom']=StudentANO['PrenomANO_ETU']

#on supprime les noms et les prénoms qui sont de trop (en fin du dataframes) et qui sont aussi duplicata des colonnes noms et prénoms
StudentANO.drop(columns=['NomANO_ETU', 'PrenomANO_ETU'], inplace=True, axis=1)

#on renomme les colonnes Nom et prénom pour ne pas avoir de doublons lors du merge qu'on va établir ensuite 
#avec le tableau PermRef
StudentANO.rename(columns={'Nom': 'NomEtudiant','Prenom': 'PrenomEtudiant'}, inplace=True)

StudentANO

,NomEtudiant,PrenomEtudiant,Nationalité,Equipe,Inscription en thèse,Sujet de thèse,Nom Dir,Prenom Dir,Nom Coenc,Prenom Coenc,Date début thèse,date fin thèse,Cotutelle,Etab cotutelle,Pays cotutelle,taux
0,Gauthier,aatami,Libanaise,Pesto,05-09-2018,Ceci est le sujet 1,Daoust,Ulyssa,Denis,Jacques,20-08-2018,2021-08-19,Non,NaN,NaN,50
1,Harvey,agrafena,Algerienne,Coast,03-04-2017,Ceci est le sujet 3,Rose,Macario,Blackburn,Gertraud,01-03-2017,2020-02-29,Non,NaN,NaN,50
2,Jutras,amyas,Iranienne,Pesto,22-10-2018,Ceci est le sujet 5,Daoust,Ulyssa,Olivier,Kasandra,01-10-2018,2021-09-30,Non,NaN,NaN,50
3,Arsenault,adisa,Bresilienne,Orpailleur,14-11-2018,Ceci est le sujet 6,Goudreau,Lisha,Turbide,Innocenzo,01-11-2018,2021-10-31,Non,NaN,NaN,50
4,Dion,adelaide,Ukrainienne,Neurorhythms,06-06-2018,Ceci est le sujet 9,Riendeau,Anto,Campbell,Callie,01-10-2017,2019-08-31,Oui,The national technical university of ukraine,Ukraine,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Brunet,aintzane,Chinoise,Larsen,07-10-2019,Ceci est le sujet 150,Couillard,Venceslao,Lheureux,Karina,01-10-2019,2022-09-30,Non,NaN,NaN,50
90,Ferland,alejo,Marocaine,Resist,28-10-2019,Ceci est le sujet 151,Lecompte,Lacy,Boulianne,Benaiah,01-10-2019,2022-09-30,Oui,Universite internationale de rabat,Maroc,30
91,Turmel,areli,Grecque,Orpailleur,24-01-2020,Ceci est le sujet 154,Goudreau,Lisha,Robertson,Bertie,01-11-2019,2022-10-31,Non,NaN,NaN,50
92,Messier,amato,Chinoise,Kiwi,31-10-2018,Ceci est le sujet 155,Caron,Breda,Jones,Ning,01-10-2018,2021-09-30,Non,NaN,NaN,50


### On a obtenu un data frame StudentANO avec les noms et prénoms des étudiants anonymisés. Nous allons dans la suite remplacer les noms et prénoms des encadrants et des coencadrants par les noms et prénoms anonymisées qui se trouvent dans le dataframa PermREF utilisé précédemment.

##### on anonymise la liste des Directeurs de thèses

In [79]:
StudentANONYME1 = pd.merge(StudentANO, PermREF, how="outer", left_on=["Nom Dir", "Prenom Dir"],right_on=["NOMEnc","PRENOMEnc"])
StudentANONYME1["Nom Dir"] = StudentANONYME1["Nom de famille"]
StudentANONYME1["Prenom Dir"] = StudentANONYME1["Prenom"]
StudentANONYME1.drop(columns=['Nom de famille', 'Prenom','NOMEnc', 'PRENOMEnc'], inplace=True, axis=1)

StudentANONYME1


,NomEtudiant,PrenomEtudiant,Nationalité,Equipe,Inscription en thèse,Sujet de thèse,Nom Dir,Prenom Dir,Nom Coenc,Prenom Coenc,Date début thèse,date fin thèse,Cotutelle,Etab cotutelle,Pays cotutelle,taux
0,Gauthier,aatami,Libanaise,Pesto,05-09-2018,Ceci est le sujet 1,Brousseau,alfonsina,Denis,Jacques,20-08-2018,2021-08-19,Non,NaN,NaN,50.0
1,Jutras,amyas,Iranienne,Pesto,22-10-2018,Ceci est le sujet 5,Brousseau,alfonsina,Olivier,Kasandra,01-10-2018,2021-09-30,Non,NaN,NaN,50.0
2,Harvey,agrafena,Algerienne,Coast,03-04-2017,Ceci est le sujet 3,Boutet,asma,Blackburn,Gertraud,01-03-2017,2020-02-29,Non,NaN,NaN,50.0
3,Awashish,aurelien,Francaise,Resist,16-05-2019,Ceci est le sujet 58,Boutet,asma,Bonneau,Apolonija,02-05-2019,2022-05-01,Non,NaN,NaN,50.0
4,Perras,babatunde,Francaise,Coast,20-07-2017,Ceci est le sujet 118,Boutet,asma,Ahmed,Morris,01-10-2017,2020-09-30,Non,NaN,NaN,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251,NaN,NaN,NaN,NaN,NaN,NaN,Labranche,banu,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
252,NaN,NaN,NaN,NaN,NaN,NaN,Labrecque,aislinn,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
253,NaN,NaN,NaN,NaN,NaN,NaN,Duguay,aletha,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN
254,NaN,NaN,NaN,NaN,NaN,NaN,Sauriol,bernardetta,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN


##### on anonymise la liste des coencadrants

In [87]:
StudentANONYME = pd.merge(StudentANONYME1, PermREF, how="outer", left_on=["Nom Coenc", "Prenom Coenc"],right_on=["NOMEnc","PRENOMEnc"])
StudentANONYME["Nom Coenc"] = StudentANONYME["Nom de famille"]
StudentANONYME["Prenom Coenc"] = StudentANONYME["Prenom"]
StudentANONYME.drop(columns=['Nom de famille', 'Prenom','NOMEnc', 'PRENOMEnc'], inplace=True, axis=1)

StudentANONYME

,NomEtudiant,PrenomEtudiant,Nationalité,Equipe,Inscription en thèse,Sujet de thèse,Nom Dir,Prenom Dir,Nom Coenc,Prenom Coenc,Date début thèse,date fin thèse,Cotutelle,Etab cotutelle,Pays cotutelle,taux
0,Gauthier,aatami,Libanaise,Pesto,05-09-2018,Ceci est le sujet 1,Brousseau,alfonsina,Carignan,aravinda,20-08-2018,2021-08-19,Non,NaN,NaN,50.0
1,Millette,alvah,Chinoise,Resist,06-12-2017,Ceci est le sujet 102,Michel,anouk,Carignan,aravinda,01-12-2017,2020-11-30,Non,NaN,NaN,50.0
2,Jutras,amyas,Iranienne,Pesto,22-10-2018,Ceci est le sujet 5,Brousseau,alfonsina,Ledoux,baldo,01-10-2018,2021-09-30,Non,NaN,NaN,50.0
3,Harvey,agrafena,Algerienne,Coast,03-04-2017,Ceci est le sujet 3,Boutet,asma,Prudhomme,anatoli,01-03-2017,2020-02-29,Non,NaN,NaN,50.0
4,Awashish,aurelien,Francaise,Resist,16-05-2019,Ceci est le sujet 58,Boutet,asma,Daviault,beavis,02-05-2019,2022-05-01,Non,NaN,NaN,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Labrecque,aislinn,NaN,NaT,NaN,NaN,NaN,NaN
401,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sauriol,bernardetta,NaN,NaT,NaN,NaN,NaN,NaN
402,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Loyer,auguste,NaN,NaT,NaN,NaN,NaN,NaN
403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sylvain,anushka,NaN,NaT,NaN,NaN,NaN,NaN


#### on obtient finalement StudentANONYME qui contient la liste des noms et prénoms anonymisées des étudiants, des encadrants, et des coencadrants

#### On redimensionne le dataframe StudentANONYME pour qu'il aie la même taille que Student

In [90]:
#on enlève les NAN contenus dans la colonne NomEtudiant
StudentANONYME = StudentANONYME[StudentANONYME['NomEtudiant'].notna()]
StudentANONYME

,NomEtudiant,PrenomEtudiant,Nationalité,Equipe,Inscription en thèse,Sujet de thèse,Nom Dir,Prenom Dir,Nom Coenc,Prenom Coenc,Date début thèse,date fin thèse,Cotutelle,Etab cotutelle,Pays cotutelle,taux
0,Gauthier,aatami,Libanaise,Pesto,05-09-2018,Ceci est le sujet 1,Brousseau,alfonsina,Carignan,aravinda,20-08-2018,2021-08-19,Non,NaN,NaN,50.0
1,Millette,alvah,Chinoise,Resist,06-12-2017,Ceci est le sujet 102,Michel,anouk,Carignan,aravinda,01-12-2017,2020-11-30,Non,NaN,NaN,50.0
2,Jutras,amyas,Iranienne,Pesto,22-10-2018,Ceci est le sujet 5,Brousseau,alfonsina,Ledoux,baldo,01-10-2018,2021-09-30,Non,NaN,NaN,50.0
3,Harvey,agrafena,Algerienne,Coast,03-04-2017,Ceci est le sujet 3,Boutet,asma,Prudhomme,anatoli,01-03-2017,2020-02-29,Non,NaN,NaN,50.0
4,Awashish,aurelien,Francaise,Resist,16-05-2019,Ceci est le sujet 58,Boutet,asma,Daviault,beavis,02-05-2019,2022-05-01,Non,NaN,NaN,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,Gordon,bengt,Francaise,Pesto,09-10-2017,Ceci est le sujet 128,Fortier,adino,Plouffe,alonso,01-10-2017,2020-09-30,Non,NaN,NaN,50.0
90,Mitchell,austen,Francaise,NaN,03-02-2016,Ceci est le sujet 136,Rondeau,alojz,Belzile,aristide,01-01-2016,2020-05-31,Non,NaN,NaN,50.0
91,Courchesne,ananias,Russe,Synalp,11-09-2017,Ceci est le sujet 139,Quenneville,basmath,Bellemare,alter,01-10-2017,2020-10-01,Non,NaN,NaN,50.0
92,Fecteau,arianne,Francaise,NaN,01-10-2018,Ceci est le sujet 146,Martinez,baltasar,Lambert,adrianne,01-10-2018,2021-09-30,Non,NaN,NaN,50.0


# FIN DU TP